In [2]:
from functions.adjust_cases_functions import prepare_cases
from functions.general_utils import  get_bool
from models.seird_model import SEIRD

import matplotlib.pyplot as plt
import scipy.io as sio
import pandas as pd
import numpy as np
import os

from global_config import config

import sys


poly_run  = 11001   #int(sys.argv[1])
name_dir  = 'bogota' #str(sys.argv[2])
drop_last_weeks = True #get_bool(sys.argv[3])

print("**** Running evaluation for hist. forecast for {}".format(name_dir))

data_dir            = config.get_property('data_dir_covid')
geo_dir             = config.get_property('geo_dir')
data_dir_mnps       = config.get_property('data_dir_col')
results_dir         = config.get_property('results_dir')
agglomerated_folder = os.path.join(data_dir, 'data_stages', 'colombia', 'agglomerated', 'geometry' )

polygons = pd.read_csv(os.path.join(agglomerated_folder, 'polygons.csv')).set_index('poly_id')
polygons = polygons.loc[poly_run]

data  =  pd.read_csv(os.path.join(agglomerated_folder, 'cases.csv'), parse_dates=['date_time'],
                    dayfirst=True).set_index('poly_id').loc[poly_run].set_index('date_time')

data  = data.resample('D').sum().fillna(0)[['num_cases','num_diseased']]
data  = prepare_cases(data, col='num_cases', cutoff=0)    # .rename({'smoothed_num_cases':'num_cases'})
data  = prepare_cases(data, col='num_diseased', cutoff=0) # .rename({'smoothed_num_cases':'num_cases'})
data  = data.rename(columns={'smoothed_num_cases': 'confirmed', 'smoothed_num_diseased':'death'})[['confirmed', 'death']]

print("**** **** Last day uploaded {}".format(pd.to_datetime(data.index.values[-1]).strftime('%Y-%b-%d')))

if drop_last_weeks:
    print("**** **** *** Droping last 2wk")
    data = data.iloc[:-14]

**** Running evaluation for hist. forecast for bogota
**** **** Last day uploaded 2021-May-08
**** **** *** Droping last 2wk


In [4]:
path_to_place = os.path.join(results_dir, 'weekly_forecast' , name_dir) #pd.to_datetime(data.index.values[-1]).strftime('%Y-%m-%d'))


In [22]:
dates_df = pd.DataFrame(data=os.listdir(path_to_place), columns=['date_frcst'])
dates_df = dates_df[dates_df['date_frcst'] != '.DS_Store']
dates_df["date_frcst"] = pd.to_datetime(dates_df["date_frcst"])
dates_df = dates_df.sort_values(by='date_frcst')
dates_df["path_to_frcst"] = dates_df["date_frcst"].map(lambda x: os.path.join(path_to_place, x.strftime('%Y-%m-%d') ) )


,date_frcst,path_to_frcst
11,2021-01-19,/Users/chaosdonkey06/Dropbox/BIOMAC/EAKF_Forec...
20,2021-01-23,/Users/chaosdonkey06/Dropbox/BIOMAC/EAKF_Forec...
10,2021-01-27,/Users/chaosdonkey06/Dropbox/BIOMAC/EAKF_Forec...
8,2021-01-29,/Users/chaosdonkey06/Dropbox/BIOMAC/EAKF_Forec...
14,2021-01-30,/Users/chaosdonkey06/Dropbox/BIOMAC/EAKF_Forec...
9,2021-02-07,/Users/chaosdonkey06/Dropbox/BIOMAC/EAKF_Forec...
0,2021-02-13,/Users/chaosdonkey06/Dropbox/BIOMAC/EAKF_Forec...
1,2021-02-14,/Users/chaosdonkey06/Dropbox/BIOMAC/EAKF_Forec...
12,2021-02-20,/Users/chaosdonkey06/Dropbox/BIOMAC/EAKF_Forec...
13,2021-02-27,/Users/chaosdonkey06/Dropbox/BIOMAC/EAKF_Forec...


In [88]:
deaths_frcst_df = pd.read_csv(os.path.join(dates_df["path_to_frcst"].iloc[1], 'deaths_df.csv' ), parse_dates=["date"])
cases_frcst_df  = pd.read_csv(os.path.join(dates_df["path_to_frcst"].iloc[1], 'cases_df.csv' ), parse_dates=["date"])

deaths_frcst_df = deaths_frcst_df[deaths_frcst_df["type"]=='forecast']
cases_frcst_df = cases_frcst_df[cases_frcst_df["type"]=='forecast']

model = SEIRD(
    confirmed = data['confirmed'].cumsum(),
    death     = data['death'].cumsum(),
    T         = len(data),
    N         = int(polygons["attr_population"]),
    samples   = mcmc_samples
    )
    
def load_samples(filename):

    x = np.load(filename, allow_pickle=True)

    mcmc_samples = x['mcmc_samples'].item()
    post_pred_samples = x['post_pred_samples'].item()
    forecast_samples = x['forecast_samples'].item()

    return mcmc_samples, post_pred_samples, forecast_samples


In [141]:
from pyro.contrib.forecast import eval_crps, eval_mae, eval_rmse

import torch

mcmc_samples, post_pred_samples, forecast_samples = load_samples(os.path.join(dates_df["path_to_frcst"].iloc[1], 'samples.npz'  ))

forecast_samples['mean_dz0'] = forecast_samples["dz0"]
forecast_samples['mean_dy0'] = forecast_samples["dy0"]
deaths_fitted = model.combine_samples(forecast_samples, f='mean_dz', use_future=True)
cases_fitted  = model.combine_samples(forecast_samples, f='mean_dy', use_future=True)
dates_frcst = deaths_frcst_df["date"]
deaths_fore = deaths_fitted[:, deaths_frcst_df.index.values]
cases_fore  = cases_fitted[:, cases_frcst_df.index.values]
data_eval = data.loc[dates_frcst]

death_samples = torch.tensor(np.array(deaths_fore)); deaths_obs = torch.tensor(list(data_eval["death"].values))
cases_samples = torch.tensor(np.array(cases_fore));  cases_obs  = torch.tensor(list(data_eval["confirmed"].values))


def compute_evals(samples_d, samples_c, obs_d, obs_c):
    
    weekdict = {'1w':6, '2w': 13, '3w': 20, '4w': 26}
    for kw in weekdict.keys()
        w = weekdict[kw]

        death_crps = eval_crps( samples_d[:w], obs_d[:w] )
        cases_crps = eval_crps( samples_c[:w], obs_c[:w] )

        death_mae = eval_mae( samples_d[:W], obs_d[:w] )
        cases_mae = eval_mae( samples_c[:W], obs_c[:w] )

        death_rmse = eval_rmse( samples_d[:w], obs_d[:w] )
        cases_rmse = eval_rmse( samples_c[:w], obs_c[:w] )



In [145]:
data_eval.iloc[27-1]


confirmed    981.0
death         17.0
Name: 2021-02-19 00:00:00, dtype: float64